# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span><ul class="toc-item"><li><span><a href="#Загрузка-и-анализ" data-toc-modified-id="Загрузка-и-анализ-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Загрузка и анализ</a></span></li><li><span><a href="#Предобработка" data-toc-modified-id="Предобработка-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Предобработка</a></span></li></ul></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span><ul class="toc-item"><li><span><a href="#Логистическая-реграссия-(LogisticRegression)" data-toc-modified-id="Логистическая-реграссия-(LogisticRegression)-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Логистическая реграссия (LogisticRegression)</a></span></li><li><span><a href="#Случайный-лес-(DecisionTreeClassifier)" data-toc-modified-id="Случайный-лес-(DecisionTreeClassifier)-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Случайный лес (DecisionTreeClassifier)</a></span></li><li><span><a href="#Стахостический-радиентный-спуск-(SGDClassifier)" data-toc-modified-id="Стахостический-радиентный-спуск-(SGDClassifier)-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Стахостический радиентный спуск (SGDClassifier)</a></span></li><li><span><a href="#Сводные-показатели" data-toc-modified-id="Сводные-показатели-2.4"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>Сводные показатели</a></span></li></ul></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

## Подготовка

### Загрузка и анализ

In [1]:
#подключение библиотек
import pandas as pd
import numpy as np

import re
from pymystem3 import Mystem
import nltk

import matplotlib.pyplot as plt

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.utils import shuffle
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings('ignore')

In [2]:
#получение данных
df = pd.read_csv('/datasets/toxic_comments.csv')
print("Общая информация:")
display(df.info())
display(df.head())
print("Количество начений в столбце toxic:")
display(df['toxic'].value_counts())
display(f"Количество дубликатов: {df.duplicated().sum()}")

Общая информация:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


None

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


Количество начений в столбце toxic:


0    143346
1     16225
Name: toxic, dtype: int64

'Количество дубликатов: 0'

In [3]:
cl_ratio = df['toxic'].value_counts()[0] / df['toxic'].value_counts()[1]
display(f"Балансировка классов: {cl_ratio}")

'Балансировка классов: 8.834884437596301'

**Первичный анализ** <br>
 - 159571 строка, 2 столбца (text, toxic);
 - дубликатов и пропусков нет;
 - классы несбалансированы (отношение 1:8,83);
 - 89% данных не токсичны

### Предобработка

In [4]:
%%time
# Подготовка файла и лемматизация текста

m = Mystem()

def lemmatize_text(text):
    text = text.lower()
    lemm_text = "".join(m.lemmatize(text))
    cleared_text = re.sub(r'[^a-zA-Z]', ' ', lemm_text) 
    return " ".join(cleared_text.split())

df['l_text'] = df['text'].apply(lemmatize_text)

df = df.drop(['text'], axis=1)

display(df.head())

,toxic,l_text
0,0,explanation why the edits made under my userna...
1,0,d aww he matches this background colour i m se...
2,0,hey man i m really not trying to edit war it s...
3,0,more i can t make any real suggestions on impr...
4,0,you sir are my hero any chance you remember wh...


CPU times: user 57.8 s, sys: 14.7 s, total: 1min 12s
Wall time: 2min 37s


In [5]:
# Разделение файла на тренировочную(70%), валидацинную(15%) и тестовую(15%) выборки

targ = df['toxic']
feat = df.drop(['toxic'], axis=1)

feat_tr, feat_v, targ_tr, targ_v = train_test_split(feat, targ, test_size=0.3, random_state=12345)
feat_v, feat_t, targ_v, targ_t = train_test_split(feat_v, targ_v, test_size=0.5, random_state=12345)

nltk.download('stopwords')
stpwrds = set(nltk_stopwords.words('english'))

count_tf_idf = TfidfVectorizer(stop_words=stpwrds)

feat_tr = count_tf_idf.fit_transform(feat_tr['l_text'].values.astype('U'))
feat_v = count_tf_idf.transform(feat_v['l_text'].values.astype('U'))
feat_t = count_tf_idf.transform(feat_t['l_text'].values.astype('U'))
print("Размеры выборок")
print(f"тренировочная: {feat_tr.shape}")
print(f"валидационная: {feat_v.shape}")
print(f"тестовая: {feat_t.shape}")

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Размеры выборок
тренировочная: (111699, 138428)
валидационная: (23936, 138428)
тестовая: (23936, 138428)


In [6]:
%%time

tr_f1 = cross_val_score(LogisticRegression(), feat_tr, targ_tr, cv=3, scoring='f1').mean()
print('F1 на CV', tr_f1)

F1 на CV 0.6832478349308616
CPU times: user 1min 6s, sys: 1min 10s, total: 2min 17s
Wall time: 2min 17s


In [7]:
%%time

# Проведем балансировку классов

tr_f1_b = cross_val_score(LogisticRegression(class_weight='balanced'), feat_tr, targ_tr, cv=3, scoring='f1').mean() 

print('F1 на CV со балансированными классами', tr_f1_b)

F1 на CV со балансированными классами 0.7463928428281513
CPU times: user 1min 4s, sys: 1min 15s, total: 2min 19s
Wall time: 2min 19s


**F1 сбалансированной модели увеличилась**

In [8]:
%%time

# Проведем ресэмплинг по нетоксичным комментариям(0)

df_tr = df.iloc[targ_tr.index]

targ_tr_zero = df_tr[df_tr['toxic'] == 0]['toxic']
targ_tr_one = df_tr[df_tr['toxic'] == 1]['toxic']

targ_tr_zero_ds = targ_tr_zero.sample(targ_tr_one.shape[0], random_state=12345)
targ_tr_ds = pd.concat([targ_tr_zero_ds, targ_tr_one])

feat_tr_ds = df.iloc[targ_tr_ds.index]
feat_tr_ds, targ_tr_ds = shuffle(feat_tr_ds, targ_tr_ds, random_state=12345)
feat_tr_ds = count_tf_idf.transform(feat_tr_ds['l_text'].values.astype('U'))

tr_f1_ds = cross_val_score(LogisticRegression(), feat_tr_ds, targ_tr_ds, cv=3, scoring='f1').mean()
print('F1 на CV с уменьшением классов', tr_f1_ds)

F1 на CV с уменьшением классов 0.8811427157190891
CPU times: user 25.8 s, sys: 32 s, total: 57.9 s
Wall time: 58 s


**Получили значительный прирост F1** 

In [9]:
# Сводная таблица полученных значений

index = ['LogisticRegression', 'LogisticRegression(balansed)', 'LogisticRegression(downsampled)']
data = {'F1 на CV':[tr_f1, tr_f1_b, tr_f1_ds]}

display(pd.DataFrame(data=data, index=index))

,F1 на CV
LogisticRegression,0.683248
LogisticRegression(balansed),0.746393
LogisticRegression(downsampled),0.881143


**Получили наилучшие показатели F1 на CV у Логистической регрессии с балансировкой по классам**, его и будем использовать в обучении

## Обучение

**Для обучения выберем следующие модели:**
 - Логистическая реграссия (LogisticRegression)
 - Случайный лес (DecisionTreeClassifier)
 - Стахостический радиентный спуск (SGDClassifier)

### Логистическая реграссия (LogisticRegression)

In [10]:
%%time

dict_classes={0:1, 1:cl_ratio}
hyperparams = [{'solver':['newton-cg', 'lbfgs', 'liblinear'], 'C':[0.1, 1, 10], 'class_weight':[dict_classes]}]

print('Настройка гипер-параметров для F1 метрики')
clf = GridSearchCV(LogisticRegression(), hyperparams, scoring='f1',cv=3)
clf.fit(feat_tr, targ_tr)
print('Лучшие параметры:')
LR_b_p = clf.best_params_
print(LR_b_p)
means = clf.cv_results_['mean_test_score']

cv_f1_LR = max(means)

Настройка гипер-параметров для F1 метрики
Лучшие параметры:
{'C': 10, 'class_weight': {0: 1, 1: 8.834884437596301}, 'solver': 'lbfgs'}
CPU times: user 6min 21s, sys: 6min 22s, total: 12min 44s
Wall time: 12min 44s


In [11]:
%%time
cl = LogisticRegression()
cl.set_params(**LR_b_p)
cl.fit(feat_tr, targ_tr)
targ_p = cl.predict(feat_v)
v_f1_LR = f1_score(targ_v, targ_p)
print('F1 на cv', cv_f1_LR)
print('F1 при валидации', v_f1_LR)

F1 на cv 0.7659187656441069
F1 при валидации 0.7722538809196305
CPU times: user 27.4 s, sys: 27.5 s, total: 54.9 s
Wall time: 55 s


### Случайный лес (DecisionTreeClassifier)

In [12]:
%%time

hyperparams = [{'max_depth':[x for x in range(30,51,2)], 'random_state':[12345], 'class_weight':[dict_classes]}]

print('Настройка гипер-параметров для F1 метрики')
clf = GridSearchCV(DecisionTreeClassifier(), hyperparams, scoring='f1',cv=3)
clf.fit(feat_tr, targ_tr)
print('Лучшие параметры:')
DTC_b_p = clf.best_params_
print(DTC_b_p)
means = clf.cv_results_['mean_test_score']

cv_f1_DTC = max(means)

Настройка гипер-параметров для F1 метрики
Лучшие параметры:
{'class_weight': {0: 1, 1: 8.834884437596301}, 'max_depth': 48, 'random_state': 12345}
CPU times: user 19min 28s, sys: 0 ns, total: 19min 28s
Wall time: 19min 37s


In [13]:
%%time
cl=DecisionTreeClassifier()
cl.set_params(**DTC_b_p)
cl.fit(feat_tr, targ_tr)
targ_p = cl.predict(feat_v)
v_f1_DTC = f1_score(targ_v, targ_p)
print('F1 на cv', cv_f1_DTC)
print('F1 при валидации', v_f1_DTC)

F1 на cv 0.6192904787372325
F1 при валидации 0.6142439024390244
CPU times: user 54.9 s, sys: 0 ns, total: 54.9 s
Wall time: 55.4 s


### Стахостический радиентный спуск (SGDClassifier)

In [14]:
%%time

hyperparams = [{'loss':['hinge', 'log', 'modified_huber'],
                'learning_rate':['constant', 'optimal', 'invscaling', 'adaptive'],
                'eta0':[0.01, 0.05, 0.1, 0.2, 0.3, 0.5], 'random_state':[12345], 'class_weight':[dict_classes]}]

print('Настройка гипер-параметров для F1 метрики')
clf = GridSearchCV(SGDClassifier(), hyperparams, scoring='f1',cv=3)
clf.fit(feat_tr, targ_tr)
print('Лучшие параметры:')
SGDC_b_p = clf.best_params_
print(SGDC_b_p)
means = clf.cv_results_['mean_test_score']

cv_f1_SGDC = max(means)

Настройка гипер-параметров для F1 метрики
Лучшие параметры:
{'class_weight': {0: 1, 1: 8.834884437596301}, 'eta0': 0.5, 'learning_rate': 'adaptive', 'loss': 'modified_huber', 'random_state': 12345}
CPU times: user 4min 29s, sys: 0 ns, total: 4min 29s
Wall time: 4min 31s


In [15]:
%%time
cl=SGDClassifier()
cl.set_params(**SGDC_b_p)
cl.fit(feat_tr, targ_tr)
targ_p = cl.predict(feat_v)
v_f1_SGDC = f1_score(targ_v, targ_p)
print('F1 на cv', cv_f1_SGDC)
print('F1 при валидации', v_f1_SGDC)

F1 на cv 0.7584178251584687
F1 при валидации 0.7592455621301776
CPU times: user 2.65 s, sys: 0 ns, total: 2.65 s
Wall time: 2.67 s


### Сводные показатели

In [16]:
index = ['Логистическая реграссия (LogisticRegression)', 'Случайный лес (DecisionTreeClassifier)',
         'Стахостический радиентный спуск (SGDClassifier)']
data = {'F1 на CV':[cv_f1_LR, cv_f1_DTC, cv_f1_SGDC],
        'F1 при валидации':[v_f1_LR, v_f1_DTC, v_f1_SGDC]}

sc_data = pd.DataFrame(data=data, index=index)
sc_data['Требование выполнено'] = sc_data['F1 при валидации'] > 0.75
display(sc_data)

,F1 на CV,F1 при валидации,Требование выполнено
Логистическая реграссия (LogisticRegression),0.765919,0.772254,True
Случайный лес (DecisionTreeClassifier),0.619290,0.614244,False
Стахостический радиентный спуск (SGDClassifier),0.758418,0.759246,True


## Выводы

В проекте произведена:
 - подготовка данных для обучения на моделях (в т.ч. проведена лемматизация);
 - определен способ баланса классов;
 - данные разделены на обучающую, валидационную и тестовою выборки;
 - обучены модели и определены лучшие для валидационной выборки.
 
В исходные данные отличаются больним количеством признаков.<br>

Хотя **Логистическая реграссия (LogisticRegression)** показала максимальный результат по **F1=0.77**, но по совокумности факторов, лучше других проявил себя **Стахостический радиентный спуск (SGDClassifier)**, выполнивнивший требование по **F1 > 0.75** и затративший минимальное **время на обучение менее 5 мин.**